IMPORTANT: Before you start, enter your name and student number below.

**Full Name**: Christina Fodtchouk

**Student Number**: 400193513

# Predictive Analytics for Nata Supermarket

Welcome to Part III of our case assignment. In this part, we will continue working with the same dataset of **Nata Supermarket**.  Our focus here will be on performing predictive modeling tasks.

Throughout this assignment, please ensure that your results are reproducible by setting the **random_state to 20**



# Loading and preparing the data

To begin with, load the data as a `pandas` data frame. Recall that you there are missing values in the data. **Make sure to address** the following issues from part I of the assignment before starting your analysis:

* Remove the missing values
* Remove any column of constant values
* Convert the column `Dt_Customer` to number of days the customer has been with the company.

In [1]:
import pandas as pd
import os
import numpy as np
from datetime import datetime
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
import plotly.express as px
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, cross_validate
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
current_working_directory = os.getcwd()
contents = os.listdir('.')
print("cwd: " , current_working_directory)
print("contents: ", contents)

# nata_dataset = pd.read_csv('./content/NATA_DATASET.csv')

df_natadataset = pd.read_excel('/content/NATA_DATASET.xlsx', sheet_name=1)

cwd:  /content
contents:  ['.config', 'NATA_DATASET.xlsx', 'sample_data']


In [3]:
num_rows, num_columns = df_natadataset.shape

print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

print(f"Number of rows including the title are {num_rows} and number columns are {num_columns}")

Number of rows: 2240
Number of columns: 29
Number of rows including the title are 2240 and number columns are 29


#####Identifying the number of rows and columns at this moment.

In [4]:
df_natadataset.isnull()

df_natadataset.isnull().sum()

,0
ID,0
Year_Birth,0
Education,0
Marital_Status,0
Income,24
Kidhome,0
Teenhome,0
Dt_Customer,0
Recency,0
MntWines,0


#####Showcasing where there are blank values, and in the table it shows Income has 24 missing values.

In [5]:
df_natadataset_copy = df_natadataset.copy()

df_natadataset_copy = df_natadataset_copy.dropna(subset=['Income'])

print(df_natadataset_copy['Income'].isnull().sum())
print(df_natadataset_copy.isnull().sum())

0
ID                     0
Year_Birth             0
Education              0
Marital_Status         0
Income                 0
Kidhome                0
Teenhome               0
Dt_Customer            0
Recency                0
MntWines               0
MntFruits              0
MntMeatProducts        0
MntFishProducts        0
MntSweetProducts       0
MntGoldProds           0
NumDealsPurchases      0
NumWebPurchases        0
NumCatalogPurchases    0
NumStorePurchases      0
NumWebVisitsMonth      0
AcceptedCmp3           0
AcceptedCmp4           0
AcceptedCmp5           0
AcceptedCmp1           0
AcceptedCmp2           0
Complain               0
Z_CostContact          0
Z_Revenue              0
Response               0
dtype: int64


#####Showing how the 24 missing values under income was removed.

In [6]:
num_rows, num_columns = df_natadataset_copy.shape

print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

print(f"Number of rows including the title are {num_rows} and number columns are {num_columns}")

Number of rows: 2216
Number of columns: 29
Number of rows including the title are 2216 and number columns are 29


#####Re-confirming the number of missing values were removed by showing the new total number of rows and then columns as simply a reference point.

In [7]:
constant_cols = [col for col in df_natadataset_copy.columns
                 if df_natadataset_copy[col].nunique() == 1]

print(constant_cols)

['Z_CostContact', 'Z_Revenue']


#####Showing which columns have constant values

In [8]:
df_natadataset_copy = df_natadataset_copy.drop(columns=constant_cols)

#####Removing the two columns with constant values.

In [9]:
print(f"{df_natadataset_copy.shape}")


(2216, 27)


#####To show the new number of columns and rows, for reference, after the two columns with constant values were removed.

In [10]:
pd.to_datetime(df_natadataset_copy['Dt_Customer'], dayfirst=True)

,Dt_Customer
0,2012-04-09
1,2014-08-03
2,2013-08-21
3,2014-10-02
4,2014-01-19
...,...
2235,2013-06-13
2236,2014-10-06
2237,2014-01-25
2238,2014-01-24


#####This is done to help identify the dates provided to show date-month-year, regardless of what order it was written in, considering the set has some dates written as year-month-day and day-month-year.

In [11]:
df_natadataset_copy2 = df_natadataset_copy.copy()

df_natadataset_copy2['Dt_Customer'] = pd.to_datetime(
    df_natadataset_copy2['Dt_Customer'],
    dayfirst=True,
    errors='coerce')

df_natadataset_copy2 = df_natadataset_copy2.dropna(subset=['Dt_Customer'])

#####To ensure that the program is able to read the dates and drop rows where date failed to convert.

In [12]:
today_date = datetime.today()

df_natadataset_copy2['Dt_Customer'] = (
    today_date - df_natadataset_copy2['Dt_Customer']).dt.days

print(df_natadataset_copy2['Dt_Customer'].head())

0    4973
1    4127
2    4474
3    4067
4    4323
Name: Dt_Customer, dtype: int64


In [13]:
current_year = datetime.today().year
df_natadataset_copy2['Age'] = current_year - df_natadataset_copy2['Year_Birth']

#####This show how the Dt_customer column values were changed to number of days the customer has been with the company by using today's date.

# Question 1 (45 points)

In this question, we will predict a customer's spending amount on each product category over a two year period. Let us assume that when we try to predict a customer's spending on a product category (such as wines), their spending on other products is not observable.

In this question and Question 2, we will focus on **wines**.

(i). **Split the data** into two data frames, X (**features**) and y (**target**).

Then, further **split the data** into **training** and **testing** sets.

In [14]:
df_nataset_new = df_natadataset_copy2.copy()

X = df_natadataset_copy2.drop(columns=['MntWines'])
y = df_natadataset_copy2['MntWines']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)

print("Training set:", X_train.shape)
print("Testing set:", X_test.shape)

Training set: (1551, 27)
Testing set: (665, 27)


#####The answer presents the number of samples in the training and test sets, I used 70% train and 30% test.

(ii). **Build a machine learning pipeline** that combines the following steps to predict spending amount on wines:

* Performing one-hot encoding for the categorical features;
* A random forest model for regression.

In the random forest model, **specify** the following hyperparameters:
* Number of trees;
* Maximum depth of any tree
* Minimum number of data points required to split a node;
* Minimum number of data points in any leaf node

In addition, **fit your model** to the training data.

In [17]:
categorical_columns = ['Education', 'Marital_Status']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)],
    remainder='passthrough')

rf_model_nata = RandomForestRegressor(
    n_estimators=200,
    max_depth=12,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=20)

pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('model', rf_model_nata)])

selected_features = [
    'Age', 'Education', 'Marital_Status', 'Income', 'Kidhome', 'Teenhome',
    'Dt_Customer', 'Recency', 'NumDealsPurchases', 'NumWebPurchases',
    'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
    'AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5',
    'Complain']

pipeline.fit(X_train[selected_features], y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Education',
                                                   'Marital_Status'])])),
                ('model',
                 RandomForestRegressor(max_depth=12, min_samples_leaf=2,
                                       min_samples_split=5, n_estimators=200,
                                       random_state=20))])

(iii). Use the model to **predict** the spending amount on wines by a customer with the following features.

| Feature |  Value  |
|---------|---------|
| Age     | 48      |
|Education|Graduation|
|Marital_Status| Married|
|Income|80,000|
|Kidhome|1|
|Teenhome|1|
|Dt-Customer|2016-10-10|
|Recency|43|     
|NumDealsPurchases|2|
|   NumWebPurchases|1|
|NumCatalogPurchases|0|
|NumStorePurchases|15|
|NumWebVisitsMonth|5|
|AcceptedCmp1,2,3,4,5| 0 |
|Complain|0|




In [18]:
customer_one = {
    'Age': 48,
    'Education': 'Graduation',
    'Marital_Status': 'Married',
    'Income': 80000,
    'Kidhome': 1,
    'Teenhome': 1,
    'Dt_Customer': (datetime.today() - datetime(2016,10,10)).days,
    'Recency': 43,
    'NumDealsPurchases': 2,
    'NumWebPurchases': 1,
    'NumCatalogPurchases': 0,
    'NumStorePurchases': 15,
    'NumWebVisitsMonth': 5,
    'AcceptedCmp1': 0,
    'AcceptedCmp2': 0,
    'AcceptedCmp3': 0,
    'AcceptedCmp4': 0,
    'AcceptedCmp5': 0,
    'Complain': 0}

customer_dataframe = pd.DataFrame([customer_one])

predicted_wines = pipeline.predict(customer_dataframe)
print("Predicted spent on wine: $", round(predicted_wines[0], 2))

Predicted spent on wine: $ 447.32


#####The answer for the model prediction the customer with such features spends on wine is about $447.32.

(iv). **Consider** **two** measures to evaluate the model's performance on the test dataset.

Based on you computational results, how would you describe the model's performance?

In [19]:
y_predicted = pipeline.predict(X_test[selected_features])
mae = mean_absolute_error(y_test, y_predicted)
print("Mean Absolute Error:", round(mae, 2))

r2 = r2_score(y_test, y_predicted)
print("R-squared:", round(r2, 3))

Mean Absolute Error: 87.3
R-squared: 0.789


#####According to the MAE, the error is about +/- 87.3 which indicates that the wine spending predictions are about $87 away from the true values. This is a reasonable level of prediction considering the range spent on wine, which goes from 0-1500 or so.

#####With respect to the R2, a value of 0.789 means that the model explains about 79% of the variation in the consumer's wine spending, which shows a strong predicive capabilities and overall good fit.




(v). **Perform** a 6-fold cross validation with a performance score of your choice.

**Note**: You may need to research on how to specify the performance score for regression models.

In [20]:
X_selected = X_train[selected_features]

cv_scores = cross_val_score(
    pipeline,
    X_selected,
    y_train,
    cv=6,
    scoring='r2')

print(cv_scores)
print(round(cv_scores.mean(), 3))

[0.80561036 0.76651126 0.83959206 0.77186916 0.77685997 0.80927925]
0.795


#####Considering the result and range of CV scores, the model seems to perform fairly well across all folds, always explaining between 76%-84% of the variance in wine spending. This means the model has strong and stable predictive capabilities.

#####With a CV mean of 79.5%, on average, the model explains about 80% of the variation in wine spending across multiple resamples of the data and this, once again, suggests that the model is performing well.

(vi). **Perform** hyperparameter tuning using `GridSearchCV` for the following hyperparameters:

* Number of trees: 50, 100
* Maximum depth of any tree: 5, 10, 15
* Minimum number of data points required to split a node: 3, 6
* Minimum number of data points in any leaf node: 2,4,8


Based on your computational result, **show**:
* the best hyperparameter comination
* the corresponding performance score

In addition, **retrieve** the best model (the one corresponding to the best performance score).

In [21]:
param_grid = {
    'model__n_estimators': [50, 100],
    'model__max_depth': [5, 10, 15],
    'model__min_samples_split': [3, 6],
    'model__min_samples_leaf': [2, 4, 8]}

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=1)

grid_search.fit(X_train[selected_features], y_train)

print(grid_search.best_params_)
print(round(grid_search.best_score_, 3))

best_model = grid_search.best_estimator_

Fitting 5 folds for each of 36 candidates, totalling 180 fits
{'model__max_depth': 15, 'model__min_samples_leaf': 2, 'model__min_samples_split': 3, 'model__n_estimators': 100}
0.792


#####This approach was able to identify that the best-performing RF configuration with 100 trees, a max depth of 15, min split size of 3, and min leaf size of 2, achieved a cross-validated R2 of 79.2%. This means that such as combination provided the strongest predictive performance among all tested models.

# Question 2 (24 points)

In this question, we will compare the performance of the best model found through `GridSearchCV` in Question 1 with the performance of the linear regression model.

(i). **Construct** a linear regression model using all relevant features and fit it to the training data.

Further, **evaluate** the model's performance on the test data and compare it with the best random forest model found in Question 1, with respect to the two performance considered in Question 1.

**Note**: You may use the same training and testing datasets as in Question 1.

In [22]:
categorical_columns = ['Education', 'Marital_Status']

preprocessor_LR = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)], remainder='passthrough')

LR_pipeline = Pipeline([
    ('preprocess', preprocessor_LR),
    ('model', LinearRegression())])

LR_pipeline.fit(X_train[selected_features], y_train)

y_pred_LR = LR_pipeline.predict(X_test[selected_features])

mae_LR = mean_absolute_error(y_test, y_pred_LR)
r2_LR = r2_score(y_test, y_pred_LR)

print("Linear Regression Performance on Test Set:")
print("MAE:", round(mae_LR, 2))
print("R²:", round(r2_LR, 3))

y_pred_RF_best = best_model.predict(X_test[selected_features])

mae_RF = mean_absolute_error(y_test, y_pred_RF_best)
r2_RF = r2_score(y_test, y_pred_RF_best)

print("\nBest Random Forest Performance on Test Set:")
print("MAE:", round(mae_RF, 2))
print("R²:", round(r2_RF, 3))

if r2_RF > r2_LR:
    print("\nRF performs better than LR on the test set.")
else:
    print("\nLR performs better or similar to RF on the test set.")

Linear Regression Performance on Test Set:
MAE: 118.13
R²: 0.694

Best Random Forest Performance on Test Set:
MAE: 87.18
R²: 0.79

RF performs better than LR on the test set.


#####The LR model this time got an MAE of 118.13 and an R2 of 0.694, which means it had a weaker predictive accuracy and a lower ability to explain variability in wine spending. In comparison to question 1, the best rRF model performed a lot better, with an MAE of 87.18 and R2 of 0.79. This means it produces more accurate predictions and addresses more underlying patterns in the data than this most recent model.

(ii). Let's further compare the distribution of prediction errors by the two models in the following steps.

**Step 1**. For both the linear regression and the (best) random forest model, compute the absolute residual residual for each record in the test dataset.
  * Note that the absolute residual is distance between the predicted value and actual value, i.e., $|y_{pred}-y_{test}|$.

So we end up with two sets of absolute residuals (one by the linear regression model and the other by the random forest model).

**Step 2**. For each pair of absolute residuals for the same test data point, we can define a point in a scatterplot. Genereate such a scatterplot using `plotly.express` (LR residuals vs. RF residuals).  

**Step 3**. Add a 45 degree reference line to the plot. This can be done using the following codes. (You may need to change `min_val` and `max_val` for better visualization).

```
min_val = 0
max_val = 10
fig.add_shape(
    type="line",
    x0=min_val, y0=min_val,
    x1=max_val, y1=max_val,
    line=dict(color="red", width=2, dash="dash"),
)
```

**Implement the above steps**.

Note that the above steps essentially creates a [Q-Q plot](https://en.wikipedia.org/wiki/Q%E2%80%93Q_plot). How would you interpret the plot (for comparing the two predictive models).

In [23]:
abs_residuals_LR = abs(y_test - y_pred_LR)
abs_residuals_RF = abs(y_test - y_pred_RF_best)

residuals_df = pd.DataFrame({
    'LR_Residual': abs_residuals_LR,
    'RF_Residual': abs_residuals_RF})

fig = px.scatter(
    residuals_df,
    x='LR_Residual',
    y='RF_Residual',
    labels={'LR_Residual':'LR Residual', 'RF_Residual':'RF Residual'},
    title='Comparison of Absolute Residuals: LR vs RF')

min_val = 0
max_val = residuals_df[['LR_Residual','RF_Residual']].max().max()
fig.add_shape(
    type="line",
    x0=min_val, y0=min_val,
    x1=max_val, y1=max_val,
    line=dict(color="red", width=2, dash="dash"),)

fig.show()

#####“The scatterplot shows that more than half the points lie below the 45-degree reference line, which means the RF model somewhat produces smaller absolute residuals than LR for the same customers. The grouping/cluster of points near the origin of the x and y axis, indicates that both models predict fairly well for many observations, however, the fact that other points disperse after and creates a wider spread of LR residuals, reaching values as high as 800, shows that LR makes larger and more frequent big errors, but the RF is able to maintain more stable and consistently lower errors across the test set.

# Question 3. (24 points)

In this question, we will consider a classification problem on customers' spendings on meat products.


(i). For the column that represents customers' spendings on meat products, **calculate** the 33.33% and 66.67% percentiles. (**Hint**: You may use the function `df['MntMeatProducts'].quantile([1/3,2/3])`.)

Based on the two percentiles, **label** each row in the dataset
* If a customer's spending is below the 33.33% percentile, label their spending as "low";

* If a customer's spending is above the 66.67% percentile, label their spending as "high";


* If a customer's spending is between the two percentiles, label their spending as "medium".

In [24]:
quantiles = df_natadataset_copy2['MntMeatProducts'].quantile([1/3, 2/3])
q33 = quantiles[1/3]
q66 = quantiles[2/3]

print("33.33% percentile:", q33)
print("66.67% percentile:", q66)

def meat_label(x):
    if x < q33:
        return 'low'
    elif x > q66:
        return 'high'
    else:
        return 'medium'

df_natadataset_copy2['MeatSpendingCategory'] = df_natadataset_copy2['MntMeatProducts'].apply(meat_label)

df_natadataset_copy2[['MntMeatProducts', 'MeatSpendingCategory']].head(10)

33.33% percentile: 24.0
66.67% percentile: 146.33333333333258


,MntMeatProducts,MeatSpendingCategory
0,546,high
1,6,low
2,127,medium
3,20,low
4,118,medium
5,98,medium
6,164,high
7,56,medium
8,24,medium
9,6,low


#####The 33.33% percentile in this case is about 24, and the 66.67% percentile is about 146. This indicates that one third of customers spend less than $24 on meat products and about one third of consumers spend more than $146.

(ii). **Build a K-nearest-neighbors model** to predict the label of a customer's spending on meat products. The model should be part of a machine learning pipeline that preprocesses the data.





In [25]:
selected_features = [
    'Age', 'Education', 'Marital_Status', 'Income', 'Kidhome', 'Teenhome',
    'Dt_Customer', 'Recency', 'NumDealsPurchases', 'NumWebPurchases',
    'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
    'AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5',
    'Complain']

X = df_natadataset_copy2[selected_features]
y = df_natadataset_copy2['MeatSpendingCategory']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=20)

categorical_columns = ['Education', 'Marital_Status']
numeric_columns = [col for col in selected_features if col not in categorical_columns]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_columns),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)])

knn_model = KNeighborsClassifier(n_neighbors=5)

pipeline_knn = Pipeline([
    ('preprocess', preprocessor),
    ('knn', knn_model)])

pipeline_knn.fit(X_train, y_train)

y_pred_knn = pipeline_knn.predict(X_test)

print(confusion_matrix(y_test, y_pred_knn))
print(classification_report(y_test, y_pred_knn))

[[131   1  29]
 [  0 118  12]
 [ 20  32 101]]
              precision    recall  f1-score   support

        high       0.87      0.81      0.84       161
         low       0.78      0.91      0.84       130
      medium       0.71      0.66      0.68       153

    accuracy                           0.79       444
   macro avg       0.79      0.79      0.79       444
weighted avg       0.79      0.79      0.79       444



#####This model was able to get an overall accuracy of about 79%, which means it predicts customers’ meat spending categories, outlined before, correctly for majority of the time. The confusion matrix and classification report illustrate how the model performs best at identifying low and high spenders, while predictions for the medium group are less accurate.

(iii). Evaluate the performace of your model in part (ii) on the test data by **generating the classification report**.

Further, **interpret** each number in the classification report based on the current context.

In [26]:
y_pred_knn = pipeline_knn.predict(X_test)
report = classification_report(y_test, y_pred_knn, target_names=['low', 'medium', 'high'])
print(report)

              precision    recall  f1-score   support

         low       0.87      0.81      0.84       161
      medium       0.78      0.91      0.84       130
        high       0.71      0.66      0.68       153

    accuracy                           0.79       444
   macro avg       0.79      0.79      0.79       444
weighted avg       0.79      0.79      0.79       444



#####In this report, the precision portrays how often the model's predictions for a given class were correct, so here the model predicted low by being correct 87% of the time. For recall, it measured how well the model is able to find all true cases of each class, so here it correctly identified about 81% of all actual low spenders. The in terms of the f1-score, it encorporates precision and recall into a single measure of overall performance for each class.

#####Overall, in this given context, the model performs best on low and medium spenders, both with f1-scores of 0.84, which means these categories are predicted pretty reliably and reasonable. For higher spenders, the model's performance is weaker, with an f1 score of 0.68. This shows the model has greater difficulty to correctly identify customers with high meat spending amounts. The overall accuracy is 79%, which shows that across all categories, the model predicts the correct spending label about four out of five times. The macro and weighted averages also being around 0.79 indicate that the model performs consistently across classes without major bias toward any single category.

# Describe how you used Gen. AI. in this assignment (2 points)

AI was used to help explain some codes when I got confused as to the deep meaning and purpose for each part of the code as well as the why certain steps are taken. Google colab had also suggested a few codes for me as I was typing, and when relevant and correct, I used its suggestion.



**Note**: The remaining 5 points will be assigned to readability of the work.